In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#import tensorflow as tf  # Version 1.0.0 (some previous versions are used in past commits)
#from tensorflow.keras import backend
from tensorflow.python.framework import ops
ops.reset_default_graph()
import tensorflow as tf
from sklearn import metrics
import random, time, os, json, glob, re, time
from random import randint
import pandas as pd
from os.path import join
import datetime

% matplotlib inline
from usefull_function import get_all_labels, get_class, get_id_label_dict

UsageError: Line magic function `%` not found.


In [3]:
import numpy as np
import pandas as pd
import re, json, glob



def get_id_label_dict(train_labels,val_labels):
    labels = pd.read_csv(train_labels, index_col=0, sep=';', header=None)
    labels = json.loads(labels.to_json())['1']
    validation_labels = pd.read_csv(val_labels, index_col=0, sep=';', header=None)
    validation_labels = json.loads(validation_labels.to_json())['1']
    labels.update(validation_labels)
    #print(labels)
    return labels

def get_all_labels(all_labels):
    df = pd.read_csv(all_labels, header = None).reset_index()
    label_class_dict = dict(df[[0,'index']].values)
    #print(label_class_dict)
    return label_class_dict



def get_class(file, id_label_dict, label_class_dict):
    f_id = re.findall(r'([0-9]*big).(npy|json)', file)[0][0]
    f_id = f_id[:-3]
    #print(f_id)
    f_label = id_label_dict[f_id]
    f_class = label_class_dict[f_label]
    #print("cl", f_class)
    return f_class


def make_numpy_array(file, db_type):
    if db_type == 'all':
        result = np.zeros((37,120))
    elif db_type == 'body':
        result = np.zeros((37,36))
    elif db_type == 'hands':
        result = np.zeros((37,84))
  
    for k in sorted(list(file['data'].keys())):
        k_int = int(k)
        try:
            data = file['data'][k][0]

            if db_type == 'all':
                bla = np.r_[data['pose_keypoints'], data['hand_left_keypoints'], data['hand_right_keypoints']]
            if db_type == 'body':
                bla = data['pose_keypoints']
            if db_type == 'hands':
                bla = np.r_[data['hand_left_keypoints'], data['hand_right_keypoints']]

            bla = bla[[False if i %3 ==2 else True for i in range(len(bla))]]
            result[k_int-1] = bla
        except:
            None
    return result



Some general information we need to handle the data

In [4]:
# How many frames there are in a video sequence, we feed to the RNN
n_steps = 37

# How many different classes there are.
n_classes = 4

# Which dataset are we looking at, choices body | hands | all
db_type = 'body'

# Where are the csv stored.
train_labels = '../labels/train.csv'
val_labels = '../labels/validation.csv'
all_labels = '../labels/labels.csv'


labels = get_id_label_dict(train_labels, val_labels)
#print(labels)
label_class_dict = get_all_labels(all_labels)
#print(label_class_dict)
file_path = join('../train/',db_type,'*.npy')
#print(file_path)
files = glob.glob(file_path)


# Loading the data, shuffling and splitting

Note we can save the data in a numpy array since our dataset is not too big. 
Otherwise we need to work with an iterator that gets the array iteratively

In [5]:
y_data = np.array([get_class(file, labels, label_class_dict) for file in files])
print(y_data)
y_original = y_data
# Make a one hot encoding of the labels
y_tmp = np.zeros((len(y_data), n_classes), dtype = np.int32)
for i in range(len(y_data)):
    y_tmp[i,y_data[i]] = 1

y_data = y_tmp
  
X_data = np.r_[[np.load(files[i]) for i in range(len(files))]]

[2 3 3 0 2 2 0 1 3 0 2 2 1 1 2 1 1 2 1 2 2 0 0 3 0 1 0 3 3 0 2 1 3 1 2 0 0
 3 1 0 2 0 3 1 0 3 3 1 2 1 1 2 0 1 2 1 0 2 1 3 2 2 0 1 2 3 0 3 3 3 0 3 0 1
 0 3 0 0 1 3 2 2 0 1 3 2 3 1 0 3 1 1 1 1 3 2 1 0 1 3 2 3 0 3 2 2 3 3 0 2 2
 0 2 1 0 2 3 3 0 3 3]


In [6]:
train_size = 120


np.random.seed(42)
permute = np.random.permutation(y_data.shape[0])
y_data = y_data[permute]
X_data = X_data[permute]

y_train, y_test = y_data[:train_size], y_data[train_size:]
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_test_true = y_original[permute][train_size:] 
permute[:10]

array([ 44,  47,   4,  55,  26,  64,  73,  10,  40, 108])

In [7]:
permute[:10]

array([ 44,  47,   4,  55,  26,  64,  73,  10,  40, 108])

# Graph 

## Some general specifications

In [8]:
# Input Data 
ops.reset_default_graph()

# Some graph specific numbers
training_data_count = len(X_train)  # 4519 training series (with 50% overlap between each serie)
test_data_count = len(X_test)  # 1197 test series
n_input = len(X_train[0][0])  # num input parameters per timestep
n_hidden = [90,50] # Hidden layer num of features



# calculated as: decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)
decaying_learning_rate = True
learning_rate = 0.0025 #used if decaying_learning_rate set to False
init_learning_rate = 0.002
decay_rate = 0.96 #the base of the exponential in the decay
decay_steps = 100000 #used in decay every 60000 steps with a base of 0.96


global_step = tf.Variable(0, trainable=False)
lambda_loss_amount = 0.0015

epochs = 50
training_iters = training_data_count *epochs  # Loop number of epoch through the data set.
batch_size = 512
display_iter = batch_size*64 # To show test set accuracy during training


In [9]:
def extract_batch_size(_train, _labels, _unsampled, batch_size):
    # Return a shuffled batch
    
    index = np.random.choice(_unsampled,size = batch_size)
    batch_s = _train[index]
    batch_labels = _labels[index]
    _unsampled = np.delete(_unsampled, index)
    
    return batch_s, batch_labels, _unsampled




----
## The graph definition

----

In [10]:
# Graph input/output
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.int32, [None, n_classes])
#y_hot = tf.one_hot(tf.cast(y,tf.int32),n_classes)


with tf.name_scope('Input') as _:
    xt = tf.transpose(x, [1, 0, 2])  # permute n_steps and batch_size
    xr = tf.reshape(xt, [-1, n_input])  
  # Rectifies Linear Unit activation function used
    
with tf.name_scope('First_fc_layer') as _:
    y_1 = tf.layers.dense(xr, n_hidden[0], 
    bias_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                  activation=tf.nn.relu, name='layer_1')
    
  # So the reader might be wondering why we take the transpose before the reshape
  # This has to do with how the lstm wants to eat the data.
  # the lstm want n_frames tensors, where each tensor is of size batchsize x num_features
  # So basically first all the 1st frames then all the 2nd frames etc.
    _X_split = tf.split(y_1, n_steps, 0) 

    
with tf.name_scope('LSTM') as _:
  # Define two stacked LSTM cells (two recurrent layers deep) with tensorflow
    lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(n_hidden[0], forget_bias=1.0, state_is_tuple=True)
    lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(n_hidden[0], forget_bias=1.0, state_is_tuple=True)
    lstm_cells = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], state_is_tuple=True)
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cells, _X_split, dtype=tf.float32)
    
  # A single output is produced, in style of "many to one" classifier, refer to http://karpathy.github.io/2015/05/21/rnn-effectiveness/ for details
    lstm_last_output = outputs[-1]
    
    
with tf.name_scope('Second_fc_layer') as _:
    y_2 = tf.layers.dense(lstm_last_output, n_hidden[1], 
                          bias_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                          kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                          activation=tf.nn.relu, name='layer_2')
    
    
with tf.name_scope('Last_fc_layer') as _:
    out = tf.layers.dense(y_2, n_classes, 
                                  bias_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                                  kernel_initializer=tf.truncated_normal_initializer(stddev=0.1), 
                                  activation=None, 
                                  name='final_layer')


with tf.name_scope('Output') as _:
    pred = out

with tf.name_scope('Optimizer') as _:
  # Loss, optimizer and evaluation
  # The regularization term
    l2 = lambda_loss_amount * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables()) 
  
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                          labels=y, 
                          logits=pred)) + l2 # Softmax loss
    if decaying_learning_rate:
        learning_rate = tf.train.exponential_decay(init_learning_rate, global_step*batch_size, decay_steps, decay_rate, staircase=True)


  #decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps) #exponentially decayed learning rate
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost,global_step=global_step) # Adam Optimizer
    tf.summary.scalar('cost', cost)
  
  
with tf.name_scope('Accuracy') as _:
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
saver = tf.train.Saver()

---

# Training of the Graph

## Only run this part if you want to train your own model

---

In [11]:
# Inititialize writers
time_now = datetime.datetime.strftime(datetime.datetime.now(), '%H%M%S')
summary_dir = './logs/'+db_type+time_now

merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(summary_dir + '/train')
test_writer=  tf.summary.FileWriter(summary_dir + '/test')

In [12]:
#sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()

# Perform Training steps with "batch_size" amount of data at each loop. 
# Elements of each batch are chosen randomly, without replacement, from X_train, 
# restarting when remaining datapoints < batch_size
step = 1
best_test_acc = 0
time_start = time.time()
unsampled_indices = list(range(0,len(X_train)))
start = time.time()

with tf.Session() as sess:
    sess.run(init)
    sess.graph.finalize()
    writer = tf.summary.FileWriter(summary_dir, sess.graph)
    while step * batch_size <= training_iters:
        if len(unsampled_indices) < batch_size:
            unsampled_indices = list(range(0,len(X_train))) 
        batch_xs, batch_ys, unsampled_indicies = extract_batch_size(X_train, y_train, unsampled_indices, batch_size)
    
    # Fit training using batch data
        _, loss_trn, acc_trn = sess.run([optimizer, cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
    
        summary = sess.run(merged, feed_dict={x: batch_xs, y: batch_ys}) 
        train_writer.add_summary(summary, step)
    
    
    # Evaluate network only at some steps for faster training: 
        if (step*batch_size % display_iter == 0) or (step == 1) or (step * batch_size > training_iters):
        
        # Evaluation on the test set (no learning made here - just evaluation for diagnosis)
            loss_tst, acc_tst = sess.run([cost, accuracy], feed_dict={x: X_test, y: y_test})
            summary = sess.run(merged, feed_dict={x: X_test, y: y_test})
        
            test_writer.add_summary(summary, step)
        
        
            print('\r Iter: {},  Accuracy test: {}, Accuracy train: {}, time: {}'.format(step*batch_size, acc_tst, acc_trn, time.time()-start))
        
            if(acc_tst > best_test_acc):
                print("\tbetter network stored,", acc_tst, ">", best_test_acc)
                best_test_acc = acc_tst
                saver.save(sess=sess, save_path=summary_dir + '/bestNetwork')
          
        
            start = time.time()
        step += 1
    print("Optimization Finished!")

  # Accuracy for test data
  
    saver.restore(sess=sess, save_path=summary_dir + '/bestNetwork')
    print("best training accuracy:", sess.run(accuracy, feed_dict={x: X_train, y: y_train}),
          "best test accuracy: ", sess.run(accuracy, feed_dict={x: X_test, y: y_test}))

time_stop = time.time()
print(("TOTAL TIME:  {}".format(time_stop - time_start)))

 Iter: 512,  Accuracy test: 1.0, Accuracy train: 0.255859375, time: 2.966365337371826
	better network stored, 1.0 > 0
Optimization Finished!
INFO:tensorflow:Restoring parameters from ./logs/body234307/bestNetwork
best training accuracy: 0.24166666 best test accuracy:  1.0
TOTAL TIME:  10.493135690689087
